In [ ]:
# default_exp core

# module name here

> API details.

In [3]:
#hide
from nbdev.showdoc import *

In [5]:
from nbdev.export import *

In [6]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_Virtual_data_setup.ipynb.
Converted index.ipynb.
